# Motivation
论文原文：[《AutoInt: Automatic Feature Interaction Learning via Self-Attentive Neural Networks》](https://arxiv.org/abs/1810.11921)

参考学习地址：[AutoInt：基于Multi-Head Self-Attention构造高阶特征](https://zhuanlan.zhihu.com/p/60185134)

不同于DCN和xDeepFM分别使用Cross层和CIN层来显式构建高阶特征，AutoInt通过Multi-head Self-Attention机制来显式构造高阶特征。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/autoint.png" width = "600" height = "500" alt="AutoInt" align=center />

 ## Embedding
 AutoInt主要对one-hot，multi-hot和dense特征都进行了embedding。
 * 对于one/multi-hot，$e_i=\frac{1}{q}V_ix_i$，其中$V_i$代表第i个field（特征组）的embedding矩阵，$x_i$为one/multi-hot表示，$q$代表每个field内命中的个数（e.g. one-hot为1），也即做了Average-Pooling；
 * 对于dense，$e_m=v_mx_m$，其中$v_m$为第m个field的embedding向量，$x_m$为scalar用作相乘

 ## Interacting Layer

每个Attention head都对应着三个转换矩阵：$\mathbf{W}_{Q},\mathbf{W}_{K},\mathbf{W}_{V} \in \R^{d'*d}$，对第$h$个head，当第m个嵌入向量$e_m$作为query时，输出为：
$$
\begin{align}
\tilde{\mathbf{e}}_m^{(h)}=\Sigma_{k=1}^{M}\alpha_{m,k}^{(h)}(\mathbf{W}_Q^{(h)}\mathbf{e}_k)
\end{align}
$$

其中$\alpha_{m,k}^{(h)}=\frac{exp(\phi^{(h)}(\mathbf{e}_m,\mathbf{e}_k))}{\Sigma_{l=1}^{M}exp(\phi^{(h)}(\mathbf{e}_m,\mathbf{e}_l))}$（i.e. softmax），$\phi^{(h)}(\mathbf{e}_m,\mathbf{e}_k)=<\mathbf{W}_Q^{(h)}\mathbf{e}_m,\mathbf{W}_K^{(h)}\mathbf{e}_k>$。式中$\phi(.)$是可选的相似度计算函数，文中简单使用了向量内积。每个向量$\mathbf{e}$都有一次作为query的机会，从而学习到在这个head下的新表达$\tilde{\mathbf{e}}$。

对第m个嵌入$\mathbf{e}_m$，简单拼接其在$H$个head的输出，然后引入标准的残差连接作为最终输出。
$$
\begin{align}
\tilde{\mathbf{e}}_m&=\tilde{\mathbf{e}}_m^{(1)}\oplus \tilde{\mathbf{e}}_m^{(2)}\oplus ...\oplus \tilde{\mathbf{e}}_m^{(H)}\in \R^{d'H} \text{, where } d代表embeddingg前空间，d'为embedding\ size\\
\tilde{\mathbf{e}}_m^{Res} &= ReLU(\tilde{\mathbf{e}}_m+\mathbf{W}_{Res}*{\mathbf{e}}_m) \text{, where } \mathbf{W}_{Res}\in \R^{d'H*d}\\
\hat{y}&=\sigma(\mathbf{w}^T({\mathbf{e}}_1^{Res}\oplus {\mathbf{e}}_2^{Res}\oplus ...\oplus {\mathbf{e}}_3^{Res})+b)\text{, where } \mathbf{w}\in \R^{d'HM} \text{, } \sigma(.)表示sigmoid函数
\end{align}
$$

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/autoint-interact.png" width = "600" height = "500" alt="AutoInt" align=center />

Tips：AutoInt没有序列数据，所以使用的是不带位置信息的MHA。